In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
matplotlib.rcParams['svg.fonttype'] = 'none' # for svgs
import matplotlib.pyplot as plt
from matplotlib import cm as cmap
import seaborn as sns
from pathlib import Path
import flexiznam as flz
from cottage_analysis.analysis import spheres, common_utils
from cottage_analysis.pipelines import pipeline_utils
from cottage_analysis.plotting import basic_vis_plots
from v1_depth_map.figure_utils import depth_selectivity, get_session_list, depth_decoder, closed_loop_rsof
from v1_depth_map.figure_utils import common_utils as plt_common_utils
from scipy import stats
from tqdm import tqdm

In [ ]:
project = "hey2_3d-vision_foodres_20220101"
flexilims_session = flz.get_flexilims_session(project)
READ_VERSION = 10
VERSION = 10
READ_ROOT = flz.get_data_root("processed", flexilims_session=flexilims_session) / "v1_manuscript_2023"/f"ver{READ_VERSION}"
SAVE_ROOT = flz.get_data_root("processed", flexilims_session=flexilims_session) / "v1_manuscript_2023"/f"ver{VERSION}"
SAVE_ROOT.mkdir(parents=True, exist_ok=True)

reload = False

In [ ]:
# LOAD DATA
# Load an example session
project = "hey2_3d-vision_foodres_20220101"
session_name = "PZAH8.2h_S20230116"
flexilims_session = flz.get_flexilims_session(project)

vs_df_example, trials_df_example = spheres.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
)

neurons_ds_example = pipeline_utils.create_neurons_ds(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=None,
    conflicts="skip",
)
neurons_df_example = pd.read_pickle(neurons_ds_example.path_full)

In [ ]:
# Concatenate all sessions for closedloop rsof
models = ["gof", "grs", "gadd", "g2d", "gratio"]
model_labels = ["OF only", "RS only", "Additive", "Conjunctive", "Pure depth"]
cols = [
    "roi",
    "best_depth",
    "preferred_depth_closedloop",
    "preferred_depth_closedloop_crossval",
    "depth_tuning_popt_closedloop",
    "depth_tuning_test_rsq_closedloop",
    "depth_tuning_test_spearmanr_pval_closedloop",
    "depth_tuning_test_spearmanr_rval_closedloop",
    "preferred_RS_closedloop_g2d",
    "preferred_RS_closedloop_crossval_g2d",
    "preferred_OF_closedloop_g2d",
    "preferred_OF_closedloop_crossval_g2d",
]
cols_to_add = [
    "rsof_test_rsq_closedloop_",
    "rsof_rsq_closedloop_",
    "rsof_popt_closedloop_",
]
for model in models:
    for col in cols_to_add:
        cols.append(f"{col}{model}")

if reload:
    mouse_list = flz.get_entities("mouse", flexilims_session=flexilims_session)
    mouse_list = mouse_list[mouse_list.name.isin(["PZAH6.4b",
                    "PZAG3.4f",
                    "PZAH8.2h",
                    "PZAH8.2i",
                    "PZAH8.2f",
                    "PZAH10.2d",
                    "PZAH10.2f"])]
    results_all = plt_common_utils.concatenate_all_neurons_df(
        flexilims_session=flexilims_session,
        session_list=get_session_list.get_sessions(
            flexilims_session,
            exclude_openloop=False,
            exclude_pure_closedloop=False,
            v1_only=True,
            trialnum_min=10,
            mouse_list=mouse_list,
        ),
        filename="neurons_df.pickle",
        cols=cols,
        read_iscell=True,
        verbose=True,
    )
    results_all["preferred_RS_closedloop_crossval_g2d"] = (
        results_all["preferred_RS_closedloop_crossval_g2d"] * 100
    )
    results_all["preferred_depth_closedloop_crossval"] = (
        results_all["preferred_depth_closedloop_crossval"] * 100
    )
    results_all["preferred_OF_closedloop_crossval_g2d"] = np.degrees(
        results_all["preferred_OF_closedloop_crossval_g2d"]
    )
    results_all.to_pickle(SAVE_ROOT / "fig2" / "results_all_rsof_closedloop.pickle")
else:
    results_all = pd.read_pickle(READ_ROOT / "fig2/results_all_rsof_closedloop.pickle")

In [ ]:
EXAMPLE_ROIS = [
    402,
    86, 
]
fontsize_dict = {"title": 8, "label": 7, "tick": 5, "legend": 5}
model_labels = ["Optic\nflow", "Running\nspeed", "Additive", "Conjunctive", "RS/OF"]
results_all["preferred_depth_amplitude"] = results_all[
    "depth_tuning_popt_closedloop"
].apply(lambda x: np.exp(x[0]))

# Plot 2 example cells with depth tuning, rs tuning, of tuning, and rsof matrix
fig = plt.figure(figsize=(18 / 2.54, 21 / 2.54))
for iroi, roi in enumerate(EXAMPLE_ROIS):
    fig.add_axes([0.05, 0.88 - 0.33*iroi, 0.1, 0.08])
    depth_tuning_kwargs = dict(
        rs_thr=None,
        plot_fit=True,
        plot_smooth=False,
        linewidth=1.2,
        linecolor="royalblue",
        closed_loop=1,
        fontsize_dict=fontsize_dict,
        markersize=7,
        markeredgecolor='w',
    )
    depth_selectivity.plot_depth_tuning_curve(
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        **depth_tuning_kwargs,
    )

    closed_loop_rsof.plot_speed_tuning(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=1,
        nbins=15,
        which_speed="RS",
        speed_min=0,
        speed_max=1.5,
        speed_thr=0,
        smoothing_sd=1,
        markersize=2.5,
        linewidth=0.8,
        markeredgecolor='w',
        plot_x=0.21,
        plot_y=0.88 - 0.33*iroi,
        plot_width=0.11,
        plot_height=0.08,
        fontsize_dict=fontsize_dict,
        legend_on=False,
    )
    ylim = plt.gca().get_ylim()
    plt.xlabel("Running speed\n(cm/s)", fontsize=fontsize_dict["label"])

    closed_loop_rsof.plot_speed_tuning(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=1,
        nbins=20,
        which_speed="OF",
        speed_min=0.01,
        speed_max=1.5,
        speed_thr=0.01,
        of_min=1e-2,
        of_max=1e4,
        smoothing_sd=1,
        markersize=2.5,
        linewidth=0.8,
        markeredgecolor='w',
        plot_x=0.36,
        plot_y=0.88 - 0.33*iroi,
        plot_width=0.11,
        plot_height=0.08,
        fontsize_dict=fontsize_dict,
        legend_on=True,
        ylim=ylim,
    )
    plt.gca().set_ylabel("")
    plt.gca().set_yticklabels([])
    plt.xlabel("Optic flow speed\n(degrees/s)", fontsize=fontsize_dict["label"])

    vmin, vmax = closed_loop_rsof.plot_RS_OF_matrix(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        log_range={
            "rs_bin_log_min": 0,
            "rs_bin_log_max": 2.5,
            "rs_bin_num": 6,
            "of_bin_log_min": -1.5,
            "of_bin_log_max": 3.5,
            "of_bin_num": 11,
            "log_base": 10,
        },
        is_closed_loop=1,
        vmin=None,
        vmax=None,
        xlabel="Running speed (cm/s)",
        ylabel="Optic flow speed\n(degrees/s)",
        plot_x=0.05,
        plot_y=0.69 - 0.33*iroi,
        plot_width=0.1,
        plot_height=0.1,
        cbar_width=0.01,
        fontsize_dict=fontsize_dict,
    )
    
    # Plot 5 model fits
    for imodel, (model, model_label) in enumerate(zip(models, model_labels)):
        if imodel == 0:
            ylabel = "Optic flow speed (degrees/s)"
        else:
            ylabel = ""
        if imodel == 1:
            xlabel = "Running speed (cm/s)"
        else:
            xlabel = ""

        closed_loop_rsof.plot_RS_OF_fit(
            fig=fig,
            neurons_df=neurons_df_example,
            roi=roi,
            model=model,
            model_label=model_label,
            min_sigma=0.25,
            vmin=vmin,
            vmax=vmax,
            log_range={
                "rs_bin_log_min": 0,
                "rs_bin_log_max": 2.5,
                "rs_bin_num": 6,
                "of_bin_log_min": -1.5,
                "of_bin_log_max": 3.5,
                "of_bin_num": 11,
                "log_base": 10,
            },
            plot_x=0.18 + 0.07*imodel,
            plot_y=0.69 - 0.33*iroi,
            plot_width=0.085,
            plot_height=0.085,
            xlabel=xlabel,
            ylabel=ylabel,
            fontsize_dict=fontsize_dict,
            cbar_width=None,
        )
        if imodel > 0:
            plt.gca().set_yticklabels([])
        plt.ylabel("")

neurons_df = results_all[
    (results_all["iscell"] == 1)
    & (results_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    & (results_all["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
    & (results_all["rsof_test_rsq_closedloop_g2d"] > 0.02)
]
# Plot preferred_depth - preferred_of scatter
closed_loop_rsof.plot_2d_hist(
    fig=fig,
    neurons_df=neurons_df,
    xcol="preferred_OF_closedloop_crossval_g2d",
    ycol="preferred_depth_closedloop_crossval",
    xlabel="Preferred optic flow speed (degrees/s)",
    ylabel="Preferred virtual depth (cm)",
    plot_x=0.6,
    plot_y=0.75,
    plot_width=0.35,
    plot_height=0.25,
    aspect_equal=True,
    fontsize_dict=fontsize_dict,
    color="royalblue",
    contour_color="k",
    linewidth=1,
    s=2,
    alpha=0.4,
)

# Plot preferred_depth - preferred_rs scatter at different bins of preferred_of
closed_loop_rsof.plot_2d_hist(
    fig=fig,
    neurons_df=neurons_df,
    xcol="preferred_RS_closedloop_crossval_g2d",
    ycol="preferred_depth_closedloop_crossval",
    xlabel="Preferred running speed (cm/s)",
    ylabel="Preferred virtual depth (cm)",
    plot_x=0.6,
    plot_y=0.55,
    plot_width=0.13,
    plot_height=0.13,
    fontsize_dict=fontsize_dict,
    color="royalblue",
    contour_color="k",
    linewidth=1,
    s=1,
    alpha=0.4,
)
plt.xlabel("")

for i, (of_min, of_max) in enumerate(zip([1, 10, 100], [10, 100, 1000])):
    results_filtered = neurons_df[
        (neurons_df["preferred_OF_closedloop_crossval_g2d"] >= of_min)
        & (neurons_df["preferred_OF_closedloop_crossval_g2d"] < of_max)
    ]
    xlabel = ""
    ylabel = ""
    closed_loop_rsof.plot_2d_hist(
        fig=fig,
        neurons_df=results_filtered,
        xcol="preferred_RS_closedloop_crossval_g2d",
        ycol="preferred_depth_closedloop_crossval",
        xlabel=xlabel,
        ylabel=ylabel,
        plot_x=0.6+(i+1)%2*0.2,
        plot_y=0.55 -(i+1)//2*0.18,
        plot_width=0.13,
        plot_height=0.13,
        fontsize_dict=fontsize_dict,
        linewidth=1,
        color="royalblue",
        contour_color="k",
        s=1,
        alpha=0.4,
    )
    plt.title(f"{of_min}-{of_max} degrees/s", fontsize=fontsize_dict["label"])
    if i==1:
        plt.xlabel("Preferred running speed (cm/s)")
    
plt.savefig(SAVE_ROOT / "fig_supp_rsof.svg", bbox_inches="tight", dpi=300)

## Stats

In [ ]:
# closedloop OF vs preferred depth
n_boots = 20000
xcol=["preferred_OF_closedloop_crossval_g2d"]
ycol=["preferred_depth_closedloop_crossval"]
np.random.seed(0)
r, distribution = plt_common_utils.hierarchical_bootstrap_stats(neurons_df, n_boots, xcol, ["mouse", "session"], ycol=ycol, correlation=True)
pval = plt_common_utils.calculate_pval_from_bootstrap(distribution, value=0)
print(f"closedloop ratio vs preferred depth: r={r}, p={pval}")

In [ ]:
# closedloop RS vs preferred depth
n_boots = 20000
xcol=["preferred_RS_closedloop_crossval_g2d"]
ycol=["preferred_depth_closedloop_crossval"]
np.random.seed(0)
r, distribution = plt_common_utils.hierarchical_bootstrap_stats(neurons_df, n_boots, xcol, ["mouse", "session"], ycol=ycol, correlation=True)
pval = plt_common_utils.calculate_pval_from_bootstrap(distribution, value=0)
print(f"closedloop ratio vs preferred depth: r={r}, p={pval}")

for i, (of_min, of_max) in enumerate(zip([1, 10, 100], [10, 100, 1000])):
    results_filtered = neurons_df[
        (neurons_df["preferred_OF_closedloop_crossval_g2d"] >= of_min)
        & (neurons_df["preferred_OF_closedloop_crossval_g2d"] < of_max)
    ]
    r, distribution = plt_common_utils.hierarchical_bootstrap_stats(results_filtered, n_boots, xcol, ["mouse", "session"], ycol=ycol, correlation=True)
    pval = plt_common_utils.calculate_pval_from_bootstrap(distribution, value=0)
    print(f"closedloop ratio vs preferred depth: r={r}, p={pval}")